In [1]:
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
from utils import three_attempts_per_season, detect_shooting_slumps, get_career_3pt_pct, get_career_3pa_per_game

In [2]:
engine = create_engine("sqlite:///Data/shooting_slumps_data.db")

In [3]:
active_player_ids = pd.read_sql("ACTIVE_PLAYER_IDS", engine)
game_logs = pd.read_sql("PLAYER_GAME_LOGS", engine)
career_stats = pd.read_sql("PLAYER_CAREER_STATS", engine)

In [4]:
print(active_player_ids.columns)

Index(['PLAYER_NAME', 'PLAYER_ID'], dtype='object')


In [5]:
print(game_logs.columns)

Index(['SEASON_ID', 'Player_ID', 'Game_ID', 'GAME_DATE', 'MATCHUP', 'WL',
       'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA',
       'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF',
       'PTS', 'PLUS_MINUS', 'VIDEO_AVAILABLE'],
      dtype='object')


In [6]:
print(career_stats.columns)

Index(['PLAYER_ID', 'SEASON_ID', 'LEAGUE_ID', 'TEAM_ID', 'TEAM_ABBREVIATION',
       'PLAYER_AGE', 'GP', 'GS', 'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A',
       'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'STL',
       'BLK', 'TOV', 'PF', 'PTS', 'PLAYER_NAME'],
      dtype='object')


In [7]:
#Isolate to players that average no fewer than 250 threes a season
filtered_player_ids = []
filtered_career_stats = []
filtered_game_logs = []

for player_id in active_player_ids["PLAYER_ID"]:
    avg_3pa = three_attempts_per_season(career_stats[career_stats["PLAYER_ID"] == player_id])
    if avg_3pa != None and avg_3pa >= 250:
        filtered_player_ids.append(player_id)


filtered_career_stats = career_stats[career_stats["PLAYER_ID"].isin(filtered_player_ids)]
filtered_game_logs = game_logs[game_logs["Player_ID"].isin(filtered_player_ids)]

In [9]:
#Find all shooting slumps
slump_player_id = []
slump_length = []
slump_change_3pct = []
slump_change_3pa = []
counter = 1
for player_id in filtered_player_ids:
    print("Detecting slumps for player " + str(counter) + "/" + str(len(filtered_player_ids)))
    counter += 1
    #Get shooting slump dataframes
    player_career = filtered_career_stats[filtered_career_stats["PLAYER_ID"] == player_id]
    player_games = filtered_game_logs[filtered_game_logs["Player_ID"] == player_id]
    shooting_slumps = detect_shooting_slumps(player_games, player_career)
    
    for slump in shooting_slumps:
        slump_player_id.append(player_id)
        slump_length.append(len(slump))
        
        career_3pct = get_career_3pt_pct(player_career)
        slump_3pct = float(np.sum(slump["FG3M"]))/float(np.sum(slump["FG3A"]))
        slump_change_3pct.append(career_3pct - slump_3pct)
        
        slump_change_3pa.append(np.sum((slump["FG3A"]) / len(slump)) - get_career_3pa_per_game(player_games))

slump_data = pd.DataFrame({"PLAYER_ID": slump_player_id, "NUM_GAMES":slump_length, "DECR_3PCT": slump_change_3pct, "CHANGE_3PA": slump_change_3pa})

Detecting slumps for player 1/135
Detecting slumps for player 2/135
Detecting slumps for player 3/135
Detecting slumps for player 4/135
Detecting slumps for player 5/135
Detecting slumps for player 6/135
Detecting slumps for player 7/135
Detecting slumps for player 8/135
Detecting slumps for player 9/135
Detecting slumps for player 10/135
Detecting slumps for player 11/135
Detecting slumps for player 12/135
Detecting slumps for player 13/135
Detecting slumps for player 14/135
Detecting slumps for player 15/135
Detecting slumps for player 16/135
Detecting slumps for player 17/135
Detecting slumps for player 18/135
Detecting slumps for player 19/135
Detecting slumps for player 20/135
Detecting slumps for player 21/135
Detecting slumps for player 22/135
Detecting slumps for player 23/135
Detecting slumps for player 24/135
Detecting slumps for player 25/135
Detecting slumps for player 26/135
Detecting slumps for player 27/135
Detecting slumps for player 28/135
Detecting slumps for player 2

/Users/nehalchigurupati/Documents/ShootingSlumps/utils.py:38: RuntimeWarning: invalid value encountered in scalar divide
  span_avg_3pt = game_data.iloc[start:end+1]["FG3M"].sum() / game_data.iloc[start:end+1]["FG3A"].sum()


Detecting slumps for player 49/135
Detecting slumps for player 50/135
Detecting slumps for player 51/135
Detecting slumps for player 52/135
Detecting slumps for player 53/135
Detecting slumps for player 54/135
Detecting slumps for player 55/135
Detecting slumps for player 56/135
Detecting slumps for player 57/135
Detecting slumps for player 58/135
Detecting slumps for player 59/135
Detecting slumps for player 60/135
Detecting slumps for player 61/135
Detecting slumps for player 62/135
Detecting slumps for player 63/135
Detecting slumps for player 64/135
Detecting slumps for player 65/135
Detecting slumps for player 66/135
Detecting slumps for player 67/135
Detecting slumps for player 68/135
Detecting slumps for player 69/135
Detecting slumps for player 70/135
Detecting slumps for player 71/135
Detecting slumps for player 72/135
Detecting slumps for player 73/135
Detecting slumps for player 74/135
Detecting slumps for player 75/135
Detecting slumps for player 76/135
Detecting slumps for

In [10]:
slump_data.to_sql("SLUMP_DATA", engine, if_exists="replace", index=False)

3480